In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages

from tqdm import tqdm
import pynwb

import bluepyopt as bpop
import bluepyopt.ephys as ephys
from neuron import h
import collections
import json

In [2]:
import efel

### Patch data processing

In [3]:
file = '/media/ubuntu/sda/Patch-seq/data/Patch/601790945_icephys.nwb'

io = pynwb.NWBHDF5IO(file, 'r')
data = io.read()

In [4]:
data.stimulus.keys()

dict_keys(['data_00000_DA0', 'data_00001_DA0', 'data_00002_DA0', 'data_00003_DA0', 'data_00004_DA0', 'data_00005_DA0', 'data_00006_DA0', 'data_00007_DA0', 'data_00008_DA0', 'data_00009_DA0', 'data_00010_DA0', 'data_00011_DA0', 'data_00012_DA0', 'data_00013_DA0', 'data_00014_DA0', 'data_00015_DA0', 'data_00016_DA0', 'data_00017_DA0', 'data_00018_DA0', 'data_00019_DA0', 'data_00020_DA0', 'data_00021_DA0', 'data_00022_DA0', 'data_00023_DA0', 'data_00024_DA0', 'data_00025_DA0', 'data_00026_DA0', 'data_00027_DA0', 'data_00028_DA0', 'data_00029_DA0', 'data_00030_DA0', 'data_00031_DA0', 'data_00032_DA0', 'data_00033_DA0', 'data_00034_DA0', 'data_00035_DA0', 'data_00036_DA0', 'data_00037_DA0', 'data_00038_DA0', 'data_00039_DA0', 'data_00040_DA0', 'data_00041_DA0', 'data_00042_DA0', 'data_00043_DA0', 'data_00044_DA0', 'data_00045_DA0', 'data_00046_DA0', 'data_00047_DA0'])

In [5]:
acquisition_data = {}
count = 0

for i in data.stimulus.keys():
    stimulus = data.get_stimulus(i)
    if stimulus.data_type == 'CurrentClampStimulusSeries':
        if len(stimulus.data) in [301000, 201000, 401000]:
            acquisition = data.get_acquisition(i.split("DA")[0] + "AD0")
            stimulus_data = str(int(np.array(stimulus.data)[60000:70000].max()))

            if stimulus_data not in acquisition_data.keys():
                acquisition_data[f'{i}_{stimulus_data}'] = np.array(acquisition.data)[40000:125000]
                # plt.figure(figsize=(10, 5))
                # sns.lineplot(acquisition_data[stimulus_data])
                # sns.lineplot(np.array(stimulus.data)[40000:125000])
                # plt.title(stimulus_data)
                # plt.show()
            else:
                acquisition_data[f'{i}_{stimulus_data}_{count}'] = np.array(acquisition.data)[40000:125000]

                # plt.figure(figsize=(10, 5))
                # sns.lineplot(acquisition_data[stimulus_data])
                # sns.lineplot(np.array(stimulus.data)[40000:125000])
                # plt.title(f'{stimulus_data}_{count}')
                # plt.show()

                count += 1




In [6]:
stage0_protocol = {}

for amplitude in acquisition_data.keys():
    amplitude = int(amplitude.split("_")[3])
    stage0_protocol[f'Amplitude_{int(amplitude)}'] = {}
    stage0_protocol[f'Amplitude_{int(amplitude)}']['stimuli'] = [
        {
            'amp': amplitude / 1000,
            'amp_end': amplitude /1000,
            'delay': 200,
            'duration': 1000,
            'stim_end': 1500,
            'totduration': 3000,
            'type': 'SquarePulse'
        }
    ]

In [7]:
stage0_feature = ['voltage_base', 'steady_state_voltage', 'voltage_deflection_vb_ssse', 'sag_amplitude', 'sag_ratio1', 
                  'decay_time_constant_after_stim']

stage1_feature = ['voltage_base', 'steady_state_voltage', 'voltage_deflection_vb_ssse', 'sag_amplitude', 'sag_ratio1', 
                  'decay_time_constant_after_stim', 'Spikecount', 'mean_frequency', 'time_to_first_spike',
                  'AP_amplitude_from_voltagebase', 'AP_width', 'AHP_depth', 'adaptation_index2']

In [8]:
traces = []
for key in acquisition_data.keys():
    trace1 = {}
    trace1['V'] = acquisition_data[key]
    trace1['T'] = np.array(range(1, len(acquisition_data[key]) + 1)) / 50
    trace1['stim_start'] = [10000 /50]
    trace1['stim_end'] = [60000 / 50]
    traces.append(trace1)

In [9]:
traces_results = efel.get_feature_values(traces, stage1_feature)

In [10]:
traces_results_dict = {}
id = 0
for key in acquisition_data.keys():
    traces_results_dict[key] = traces_results[id]
    id += 1

### Model Construction


In [11]:
h.nrn_load_dll("/media/ubuntu/sda/Patch-seq/data/AdEx_Neuron/x86_64/.libs/libnrnmech.so")

1.0

In [ ]:
def create_cell_model():  
    morph = ephys.morphologies.NrnFileMorphology(
        morphology_path='simple.swc'
    )
    
    # 设置机制
    mechs = [
        ephys.mechanisms.NrnMODPointProcessMechanism(
            name='AdEx',
            suffix='AdEx',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            mod_path='/Users/jin/Desktop/adex.mod'
        )
    ]
    
    # 确保参数与adex.mod匹配
    parameters = [
        ephys.parameters.NrnPointProcessParameter(
            name='a',
            param_name='a',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[0.1, 10.0],
            value=4.0,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='b',
            param_name='b',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[0.001, 0.2],
            value=0.0805,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='tauw',
            param_name='tauw',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[50.0, 300.0],
            value=144.0,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='DeltaT',
            param_name='DeltaT',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[0.5, 5.0],
            value=2.0,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='EL',
            param_name='EL',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[-80.0, -60.0],
            value=-70.6,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='Vr',
            param_name='Vr',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[-80.0, -60.0],
            value=-70.6,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='VT',
            param_name='VT',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[-60.0, -40.0],
            value=-50.4,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='Vth',
            param_name='Vth',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[-40.0, -20.0],
            value=-30.0,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='gL',
            param_name='gL',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[10.0, 50.0],
            value=30.0,
            frozen=False
        ),
        ephys.parameters.NrnPointProcessParameter(
            name='C',
            param_name='C',
            locations=[ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic')],
            bounds=[100.0, 500.0],
            value=281.0,
            frozen=False
        )
    ]

    # 创建细胞模型
    cell_model = ephys.models.CellModel(
        name='AdEx',
        morph=morph,
        mechs=mechs,
        params=parameters
    )

    return cell_model, parameters


In [13]:
cell_model, paramerter = create_cell_model()

In [14]:
cell_model.mechanisms[0]

In [15]:
def create_protocol(amplitude=0.1, delay=100, duration=800, total_duration=1000):
    """创建电生理刺激协议"""
    # 创建注入电流刺激
    stim = ephys.stimuli.NrnSquarePulse(
        step_amplitude=amplitude,  # nA
        step_delay=delay,      # ms
        step_duration=duration,   # ms
        location=ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic'),
        total_duration=total_duration  # ms
    )
    
    # 创建记录
    rec = ephys.recordings.CompRecording(
        name='v_soma',
        location=ephys.locations.NrnSeclistLocation('soma', seclist_name='somatic'),
        variable='v'
    )
    
    # 创建协议
    ephys_protocol = ephys.protocols.SweepProtocol(
        name='step_protocol',
        stimuli=[stim],
        recordings=[rec]
    )
    
    return ephys_protocol

In [16]:
ephys_protocol = create_protocol(amplitude=0.1)


In [17]:
nrn = ephys.simulators.NrnSimulator()

In [18]:
default_params = {
    'a': 4.0,
    'b': 0.0805,
    'tauw': 144.0,
    'DeltaT': 2.0,
    'EL': -70.6
}

ephys_protocol.run(cell_model=cell_model, param_values=default_params, sim=nrn)

SweepProtocolException: Failed to run Neuron Sweep Protocol

In [ ]:
stage0_feature = ['voltage_base', 'steady_state_voltage', 'voltage_deflection_vb_ssse', 
                      'sag_amplitude', 'sag_ratio1', 'decay_time_constant_after_stim']

In [ ]:
hh_mech = ephys.mechanisms.NrnMODMechanism(
        name='hh',
        suffix='hh')

In [ ]:
hh_mech.to_dict()

{'name': 'hh',
 'comment': '',
 'mod_path': None,
 'suffix': 'hh',
 'locations': None,
 'preloaded': True,
 'class': "<class 'bluepyopt.ephys.mechanisms.NrnMODMechanism'>"}

In [ ]:
ephys.locations.NrnSeclistLocation('soma', seclist_name='soma')